<a href="https://colab.research.google.com/github/itissandeep98/StackOverFlow-Tag-Predictor/blob/master/ML_project_20k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd 
import csv,nltk,random,pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer,WordNetLemmatizer,SnowballStemmer
import re
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split,ShuffleSplit,GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.ensemble import RandomForestClassifier
import pickle
import joblib
from sklearn.tree import DecisionTreeClassifier
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# preprocess

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Ml Project/preprocessed_20k.csv')
preprocessed_df = df[["question","Tags"]]
preprocessed_df

,question,Tags
0,check upload file imag without mime type like ...,php image-processing file-upload upload mime-t...
1,prevent firefox close press ctrl favorit edito...,firefox
2,error invalid type list variabl import matlab ...,r matlab machine-learning
3,replac special charact url probabl simpl simpl...,c# url encoding
4,modifi whoi contact detail use modifi function...,php api file-get-contents
...,...,...
19976,instal interbas firebird conflict interbas fir...,firebird interbase
19977,chang text color activ non activ nstextfield w...,objective-c cocoa nstextfield nstextview
19978,necessari wsdl file creation webservic php que...,php web-services wsdl
19979,java gc major gc vs minor gc latenc sensit app...,java garbage-collection


In [ ]:
preprocessed_df["tag_count"] = preprocessed_df["Tags"].apply(lambda text: len(text.split(" ")))

In [ ]:
preprocessed_df.tag_count.value_counts()

3    5783
2    5287
4    3824
1    2728
5    2359
Name: tag_count, dtype: int64

In [ ]:
vectorizer = CountVectorizer(tokenizer = lambda x: x.split())
multilabel_y = vectorizer.fit_transform(preprocessed_df['Tags'])

In [ ]:
a=multilabel_y.sum(axis=0).tolist()[0]
print(pd.DataFrame(a).value_counts(),len(a))

1      4592
2      1464
3       706
4       446
5       284
       ... 
125       1
129       1
137       1
139       1
73        1
Length: 141, dtype: int64 8938


In [ ]:
print(multilabel_y.shape)
tags=vectorizer.get_feature_names()
print(tags)

(19981, 8938)
['.each', '.emf', '.htaccess', '.net', '.net-2.0', '.net-3.5', '.net-4.0', '.net-4.5', '.net-assembly', '.net-framework', '.net4.0', '.post', '.refresh', '.war', '.x', '1080p', '10gbethernet', '1and1', '2003', '2007', '2010', '2013', '24bit', '2d', '2d-array', '2wire', '3.0', '32-bit', '32bit', '32bit-64bit', '3d', '3d-engine', '3d-model', '3d-modelling', '3dsmax', '3g', '404-error', '4d', '5.1', '64-bit', '64bit', '6502', '6510', '68000', '7-zip', '70-667', '7zip', '802.11n', '8086', '8bit', '9-bit-serial', 'a-record', 'a-star', 'aabb', 'aapt', 'abaddressbook', 'abap', 'abelian-groups', 'aberration', 'absolute-path', 'absolute-positioning', 'absolute-value', 'abstract', 'abstract-algebra', 'abstract-class', 'abstract-data-type', 'abstract-syntax-tree', 'abstraction', 'acc', 'accelerate-framework', 'accelerated-c++', 'accelerometer', 'accented-strings', 'access', 'access-control', 'access-denied', 'access-token', 'access-vba', 'access-violation', 'accessibility', 'accesso

In [ ]:
def tags_to_consider(n):
    tag_i_sum = multilabel_y.sum(axis=0).tolist()[0]
    sorted_tags_i = sorted(range(len(tag_i_sum)), key=lambda i: tag_i_sum[i], reverse=True)
    yn_multilabel=multilabel_y[:,sorted_tags_i[:n]]
    #names=[vectorizer.get_feature_names() for i in multilabel_y[:,sorted_tags_i[:n]] ]
    return yn_multilabel
yx_multilabel = tags_to_consider(500)

In [ ]:
a=yx_multilabel.sum(axis=0).tolist()[0]
print(pd.DataFrame(a).value_counts(),len(a))

18      35
20      28
19      24
21      22
17      20
        ..
125      1
129      1
137      1
95       1
1533     1
Length: 125, dtype: int64 500


In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(preprocessed_df, yx_multilabel,stratify=yx_multilabel.todense(), test_size = 0.2,random_state = 42, )
X_train=preprocessed_df.head(15984)
X_test=preprocessed_df.tail(preprocessed_df.shape[0] - 15984)

Y_train = yx_multilabel[0:15984,:]
Y_test = yx_multilabel[15984:preprocessed_df.shape[0],:]


In [ ]:
vectorizer = CountVectorizer(min_df=0.00009, max_features=13000,  \
                             tokenizer = lambda x: x.split(), ngram_range=(1,4))
x_train_multilabel = vectorizer.fit_transform(X_train['question'])
x_test_multilabel = vectorizer.transform(X_test['question'])

# LR

In [ ]:
# param={'estimator__alpha': [10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10**0, 10**1]}
clf_lr = OneVsRestClassifier(SGDClassifier(loss='log',alpha=0.001, penalty='l1'))
# gsv = GridSearchCV(estimator = clf, param_grid=param, cv=3, verbose=0, scoring='f1_micro',n_jobs=-1)
clf_lr.fit(x_train_multilabel, Y_train)

In [ ]:
y_pred = clf_lr.predict(x_test_multilabel)
print("Accuracy :",metrics.accuracy_score(Y_test,y_pred))
print("Macro f1 score :",metrics.f1_score(Y_test, y_pred, average = 'macro'))
print("Micro f1 scoore :",metrics.f1_score(Y_test, y_pred, average = 'micro'))
print("hamming loss :",metrics.hamming_loss(Y_test,y_pred))

Accuracy : 0.167125344008006
Macro f1 score : 0.22093090004235288
Micro f1 scoore : 0.3903719120441589
hamming loss : 0.0033710282712034025


# SVM

In [ ]:
clf_svm = OneVsRestClassifier(SGDClassifier(alpha=0.001, penalty='l2'))
clf_svm.fit(x_train_multilabel, Y_train)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.001, average=False,
                                            class_weight=None,
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal',
                                            loss='hinge', max_iter=1000,
                                            n_iter_no_change=5, n_jobs=None,
                                            penalty='l2', power_t=0.5,
                                            random_state=None, shuffle=True,
                                            tol=0.001, validation_fraction=0.1,
                                            verbose=0, warm_start=False),
                    n_jobs=None)

In [ ]:
y_pred = clf_svm.predict(x_test_multilabel)
print("Accuracy :",metrics.accuracy_score(Y_test,y_pred))
print("Macro f1 score :",metrics.f1_score(Y_test, y_pred, average = 'macro'))
print("Micro f1 scoore :",metrics.f1_score(Y_test, y_pred, average = 'micro'))
print("hamming loss :",metrics.hamming_loss(Y_test,y_pred))

Accuracy : 0.19489617212909682
Macro f1 score : 0.23465788992848166
Micro f1 scoore : 0.4013377926421405
hamming loss : 0.003134851138353765


# Bernoulli Naive bayes

In [ ]:
clf_BN = OneVsRestClassifier(BernoulliNB())
clf_BN.fit(x_train_multilabel, Y_train)

OneVsRestClassifier(estimator=BernoulliNB(alpha=1.0, binarize=0.0,
                                          class_prior=None, fit_prior=True),
                    n_jobs=None)

In [ ]:
y_pred = clf_BN.predict(x_test_multilabel)
print("Accuracy :",metrics.accuracy_score(Y_test,y_pred))
print("Macro f1 score :",metrics.f1_score(Y_test, y_pred, average = 'macro'))
print("Micro f1 scoore :",metrics.f1_score(Y_test, y_pred, average = 'micro'))
print("hamming loss :",metrics.hamming_loss(Y_test,y_pred))

Accuracy : 0.11808856642481862
Macro f1 score : 0.015572751621269074
Micro f1 scoore : 0.19790132547864506
hamming loss : 0.004360270202651989


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


# Decision Tree

In [ ]:
clf_dt = OneVsRestClassifier(DecisionTreeClassifier())
clf_dt.fit(x_train_multilabel, Y_train)
pickle.dump(clf_svm,open('svm.sav','wb'))

In [ ]:
y_pred = clf_dt.predict(x_test_multilabel)
print("Accuracy :",metrics.accuracy_score(Y_test,y_pred))
print("Macro f1 score :",metrics.f1_score(Y_test, y_pred, average = 'macro'))
print("Micro f1 scoore :",metrics.f1_score(Y_test, y_pred, average = 'micro'))
print("hamming loss :",metrics.hamming_loss(Y_test,y_pred))

Accuracy : 0.15861896422316737
Macro f1 score : 0.278416687728279
Micro f1 scoore : 0.39570552147239263
hamming loss : 0.003942957217913435
